In [1]:
import sys
import os
sys.path.insert(0, "../../python")

import cProfile
from propagation import laser
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from line_profiler import LineProfiler

def do_profile(follow=[]):
    def inner(func):
        def profiled_func(*args, **kwargs):
            try:
                profiler = LineProfiler()
                profiler.add_function(func)
                for f in follow:
                    profiler.add_function(f)
                profiler.enable_by_count()
                return func(*args, **kwargs)
            finally:
                profiler.print_stats()
        return profiled_func
    return inner

In [2]:
Nx = 2**8
X = 100
x = np.linspace(-X/2, X/2, Nx, False)
Ny = 2**8
Y = 100
y = np.linspace(-Y/2, Y/2, Ny, False)

Nz = 256
Z = 100
z = np.linspace(0, Z, Nz)

w0 = 5
zR = np.pi * w0**2
wZ = w0 * np.sqrt(1 + (Z/zR)**2)
E = np.array(np.exp(-(np.reshape(x-10, (Nx, 1))**2 + np.reshape(y, (1, Ny))**2) / w0**2), dtype='complex128')
EE = (w0 / wZ) * np.exp(-(x-10)**2 / wZ**2)

cProfile.run('laser.fourier_prop2(E, x, y, z, 0.8)')

         155 function calls in 1.692 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    1.692    1.692 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _methods.py:37(_any)
        2    0.000    0.000    0.001    0.001 fftpack.py:102(fft)
        2    0.074    0.037    0.471    0.236 fftpack.py:198(ifft)
        4    0.000    0.000    0.317    0.079 fftpack.py:47(_raw_fft)
        2    0.000    0.000    0.000    0.000 fftpack.py:611(_cook_nd_args)
        2    0.002    0.001    0.475    0.238 fftpack.py:630(_raw_fftnd)
        1    0.000    0.000    0.002    0.002 fftpack.py:834(fft2)
        1    0.000    0.000    0.474    0.474 fftpack.py:924(ifft2)
        4    0.000    0.000    0.000    0.000 fftpack.py:95(_unitary)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:138(reshape)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1866(any)
        3    0.0

In [3]:
@do_profile(follow=[laser.fourier_prop2])
def test(E, x, y, z):
    laser.fourier_prop2(E, x, y, z, 0.8)
    
result = test(E, x, y, z)
# I split the function up to see which part was slow, it ends up close to two thirds of the computation is spent
# calculating the np.exp(...) part, potentially can be spead up

Timer unit: 1e-06 s

Total time: 1.34792 s
File: ../../python/propagation/laser.py
Function: fourier_prop2 at line 131

Line #      Hits         Time  Per Hit   % Time  Line Contents
   131                                           def fourier_prop2(E, x, y, z, lam, n=1):
   132                                               """ Propagates an electromagnetic wave from a 2D boundary.
   133                                           
   134                                               Uses the Rayleigh-Sommerfeld transfer function to propagate an
   135                                               electromagnetic wave from a 2D boundary. The calculation assumes a
   136                                               homogeneous index of refraction in the region of propagation.
   137                                           
   138                                               Parameters
   139                                               ----------
   140                              

In [4]:
from propagation import laserv2
#import pyximport; pyximport.install()
#from propagation import laserv2
path = '/media/robert/Data_Storage/Data/Optimization/fourier_prop/'
if not os.path.exists(path):
    os.makedirs(path)

cProfile.run('laserv2.fourier_prop2(E, x, y, z, 0.8, path=path)')
# The writing data to file seems to kill the computation speed.

         14200 function calls in 1.388 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:402(parent)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.000    0.000    1.388    1.388 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 base.py:104(_lcpl)
        1    0.000    0.000    0.000    0.000 base.py:110(_e)
        1    0.000    0.000    0.000    0.000 base.py:120(get_lcpl)
        4    0.000    0.000    0.000    0.000 base.py:237(id)
        2    0.000    0.000    0.000    0.000 base.py:270(__init__)
        1    0.000    0.000    0.000    0.000 base.py:98(_lapl)
        1    0.000    0.000    0.000    0.000 dataset.py:313(__init__)
        1    0.000    0.000    0.000    0.000 dataset.py:50(make_new_dset)
        2    0.000    0.000    0.001    0.001 fftpack.py:102(f